In [116]:
#Import spark session
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[2]").appName("HRSAssignment").getOrCreate()# change session setup according to running Env

In [119]:
from pyspark.sql import functions as F
hotelsDF=spark.read.options(header=True,delimiter=',').csv('D:\Companies\HRS Group\HRS_Assignment/hotels.csv') #change file location and name according to you
# hotelsDF.select(['market_segment']).distinct().show()
tourOpDF=hotelsDF.filter(F.col('market_segment')=='Offline TA/TO') #extracted every booking that the Tour Operators as Market Segment designations.


int_type_columns=['stays_in_weekend_nights','stays_in_week_nights','children','babies'] # consume these columns as int below
for col in int_type_columns:
    tourOpDF=tourOpDF.withColumn(col, tourOpDF[col].cast('int'))
    
# tourOpDF.printSchema()

In [123]:
import datetime
def monthNametoNumber(str):  # used to convert month name to month number eg. July->07
    result=datetime.datetime.strptime(str,'%B')
    return result.month 

monthNametoNumberUDF = F.udf(lambda z: monthNametoNumber(z),F.StringType())
arrdtDF=tourOpDF.select('*',F.to_date(F.concat_ws('-',F.col('arrival_date_year'),monthNametoNumberUDF(F.col('arrival_date_month')),F.col('arrival_date_day_of_month')) \
               ,'y-M-d').alias('arrival_date'))# Added arrival_date


depDtDF=arrdtDF.withColumn('departure_date',F.expr("date_add(arrival_date,stays_in_weekend_nights+stays_in_week_nights)"))
#added departure date

In [124]:
depDtDF.withColumn('with_family_breakfast',F.when((depDtDF.babies+depDtDF.children)>0,'Yes').otherwise('No')).show()
# added with family breakfast.

+------------+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+----+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+-------+--------------------+---------------+-----+---------------------------+-------------------------+------------------+-----------------------+------------+--------------+---------------------+
|       hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|agent|company|days_in_waiting_list|